Deep Learning
=============

Assignment 3
------------

Previously in `2_fullyconnected.ipynb`, you trained a logistic regression and a neural network model.

The goal of this assignment is to explore regularization techniques.

---
ToDo
----

1) faire varier les hyperparamètres et enregistrer les performances
- longueur du set d'entrainement
- vitesse d'apprentissage : valeur init et taux
- batch size ?
- 

2) modifier le modèle :
- ajouter des couches tant que 1) le justifie
- dropout

3) factoriser le code :
- créer une classe pour le réseau de neurones (scope + init + placeholders I/O)
- créer une fonction de compte rendu : print (si verbose), log, etc => dico des var à logger
- ébauche de framework pour la modification des hyperparamètres

---

In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from six.moves import cPickle as pickle

First reload the data we generated in _notmist.ipynb_.

In [2]:
pickle_file = 'notMNIST_cleaned.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (194661, 28, 28) (194661,)
Validation set (9967, 28, 28) (9967,)
Test set (9967, 28, 28) (9967,)


Reformat into a shape that's more adapted to the models we're going to train:
- data as a flat matrix,
- labels as float 1-hot encodings.

In [3]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
  dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
  # Map 0 to [1.0, 0.0, 0.0 ...], 1 to [0.0, 1.0, 0.0 ...]
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels

In [4]:
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (194661, 784) (194661, 10)
Validation set (9967, 784) (9967, 10)
Test set (9967, 784) (9967, 10)


In [5]:
def accuracy(predictions, labels, name='accuracy'):
    correct_predictions = tf.equal(tf.argmax(predictions,1), tf.argmax(labels,1))
    prediction_accuracy = tf.reduce_mean(tf.cast(correct_predictions, tf.float32), name=name)
    return prediction_accuracy

---
Problem 1
---------

Introduce and tune L2 regularization for both logistic and neural network models. Remember that L2 amounts to adding a penalty on the norm of the weights to the loss. In TensorFlow, you can compute the L2 loss for a tensor `t` using `nn.l2_loss(t)`. The right amount of regularization should improve your validation / test accuracy.

---

In [6]:
batch_size = 128

input_size = image_size * image_size
hidden_size = 90
output_size = num_labels

num_steps = 6000

graph = tf.Graph()
with graph.as_default():

    # Generic placeholders : it can either run with ttrain / test / valid data
    input_data = tf.placeholder(tf.float32, shape=(None, input_size), name='input')
    output_data = tf.placeholder(tf.float32, shape=(None, output_size), name='output')

    # First layer
    with tf.variable_scope('first_layer') as scope:
        layer_1_weights = tf.Variable(tf.truncated_normal([input_size, hidden_size]), name='layer_1_weights')
        layer_1_biases = tf.Variable(tf.zeros([hidden_size]), name='layer_1_biases')
        layer_1_logits = tf.matmul(input_data, layer_1_weights) + layer_1_biases
        layer_1_relu = tf.nn.relu(layer_1_logits, name='layer_1_relu')
    
    # Second layer
    with tf.variable_scope('second_layer') as scope:
        layer_2_weights = tf.Variable(tf.truncated_normal([hidden_size, output_size]), name='layer_2_weights')
        layer_2_biases = tf.Variable(tf.zeros([output_size]), name='layer_2_biases')
        layer_2_logits = tf.matmul(layer_1_relu, layer_2_weights) + layer_2_biases
    
    # Output = prediction
    with tf.variable_scope('prediction') as scope:
        model_output = tf.nn.softmax(layer_2_logits, name='model_prediction')
        model_accuracy = accuracy(model_output, output_data, name='model_accuracy')
    
    # Loss
    with tf.variable_scope('loss') as scope:
        accuracy_loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(layer_2_logits, output_data), name='loss')
        regularization_loss = tf.nn.l2_loss(layer_1_weights) + tf.nn.l2_loss(layer_1_biases) + \
                              tf.nn.l2_loss(layer_2_weights) + tf.nn.l2_loss(layer_2_biases)
        loss = accuracy_loss + 0.001 * regularization_loss
    
    # Optimizer with dynamic learning rate
    with tf.variable_scope('optimizer') as scope:
        global_step = tf.Variable(0, trainable=False)
        learning_rate = tf.train.exponential_decay(0.5, global_step, num_steps * batch_size, 0.96)
        optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)

In [7]:
with tf.Session(graph=graph, config=tf.ConfigProto(allow_soft_placement=True)) as session:
    
    # log the quality of the model
    train_accuracy_log = tf.scalar_summary("train accuracy", model_accuracy)
    valid_accuracy_log = tf.scalar_summary("validation accuracy", model_accuracy)
    test_accuracy_log = tf.scalar_summary("test accuracy", model_accuracy)
    loss_log = tf.scalar_summary("loss", loss)
    
    # Merge all the summaries and write them out to /tmp/mnist_logs
    writer = tf.train.SummaryWriter("/tmp/udacity_logs", session.graph_def)
    tf.initialize_all_variables().run()
    print("Initialized")
    
    for step in range(1,num_steps+1):
        if (step % 100 == 0):
            print("==> step %d" % step)
            
            #evaluate on the train set
            feed_dict = {input_data : train_dataset, output_data : train_labels}
            acc, acc_str = session.run([model_accuracy, train_accuracy_log], feed_dict=feed_dict)
            writer.add_summary(acc_str, step)
            print("Train set accuracy: %f" % acc)
            
            #evaluate on the validation set
            feed_dict = {input_data : valid_dataset, output_data : valid_labels}
            acc, acc_str = session.run([model_accuracy, valid_accuracy_log], feed_dict=feed_dict)
            writer.add_summary(acc_str, step)
            print("Validation set accuracy: %f" % acc)
            
            #evaluate on the test set
            feed_dict = {input_data : test_dataset, output_data : test_labels}
            acc, acc_str = session.run([model_accuracy, test_accuracy_log], feed_dict=feed_dict)
            writer.add_summary(acc_str, step)
            print("Test set accuracy: %f" % acc)
        else:
            # Generate a minibatch.
            offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
            batch_data = train_dataset[offset:(offset + batch_size), :]
            batch_labels = train_labels[offset:(offset + batch_size), :]
            feed_dict = {input_data : batch_data, output_data : batch_labels}
            session.run(optimizer, feed_dict=feed_dict)

Initialized
==> step 100
Train set accuracy: 0.696066
Validation set accuracy: 0.765225
Test set accuracy: 0.765225
==> step 200
Train set accuracy: 0.653762
Validation set accuracy: 0.705428
Test set accuracy: 0.705428
==> step 300
Train set accuracy: 0.729484
Validation set accuracy: 0.788502
Test set accuracy: 0.788502
==> step 400
Train set accuracy: 0.756500
Validation set accuracy: 0.830440
Test set accuracy: 0.830440
==> step 500
Train set accuracy: 0.761226
Validation set accuracy: 0.821210
Test set accuracy: 0.821210
==> step 600
Train set accuracy: 0.767719
Validation set accuracy: 0.841678
Test set accuracy: 0.841678
==> step 700
Train set accuracy: 0.773483
Validation set accuracy: 0.844888
Test set accuracy: 0.844888
==> step 800
Train set accuracy: 0.784132
Validation set accuracy: 0.857430
Test set accuracy: 0.857430
==> step 900
Train set accuracy: 0.782067
Validation set accuracy: 0.839771
Test set accuracy: 0.839771
==> step 1000
Train set accuracy: 0.787343
Validatio

---
Problem 2
---------
Let's demonstrate an extreme case of overfitting. Restrict your training data to just a few batches. What happens?

---

In [18]:
def random_subset(dataset, labels, size):
    subset_indices = np.random.choice(labels.shape[0],size)
    shuffled_dataset = dataset[subset_indices]
    shuffled_labels = labels[subset_indices]
    return shuffled_dataset, shuffled_labels

In [26]:
batch_size = 128

input_size = image_size * image_size
hidden_size = 90
output_size = num_labels

num_epoch = 20

graph = tf.Graph()
with graph.as_default():

    # Generic placeholders : it can either run with ttrain / test / valid data
    input_data = tf.placeholder(tf.float32, shape=(None, input_size), name='input')
    output_data = tf.placeholder(tf.float32, shape=(None, output_size), name='output')

    # First layer
    with tf.variable_scope('first_layer') as scope:
        layer_1_weights = tf.Variable(tf.truncated_normal([input_size, hidden_size]), name='layer_1_weights')
        layer_1_biases = tf.Variable(tf.zeros([hidden_size]), name='layer_1_biases')
        layer_1_logits = tf.matmul(input_data, layer_1_weights) + layer_1_biases
        layer_1_relu = tf.nn.relu(layer_1_logits, name='layer_1_relu')
    
    # Second layer
    with tf.variable_scope('second_layer') as scope:
        layer_2_weights = tf.Variable(tf.truncated_normal([hidden_size, output_size]), name='layer_2_weights')
        layer_2_biases = tf.Variable(tf.zeros([output_size]), name='layer_2_biases')
        layer_2_logits = tf.matmul(layer_1_relu, layer_2_weights) + layer_2_biases
    
    # Output = prediction
    with tf.variable_scope('prediction') as scope:
        model_output = tf.nn.softmax(layer_2_logits, name='model_prediction')
        model_accuracy = accuracy(model_output, output_data, name='model_accuracy')
    
    # Loss
    with tf.variable_scope('loss') as scope:
        accuracy_loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(layer_2_logits, output_data), name='loss')
        regularization_loss = tf.nn.l2_loss(layer_1_weights) + tf.nn.l2_loss(layer_1_biases) + \
                              tf.nn.l2_loss(layer_2_weights) + tf.nn.l2_loss(layer_2_biases)
        loss = accuracy_loss + 0.001 * regularization_loss
    
    # Optimizer with dynamic learning rate
    with tf.variable_scope('optimizer') as scope:
        global_step = tf.Variable(0, trainable=False)
        subset_size = tf.Variable(train_dataset.shape[0], trainable=False)
        learning_rate = tf.train.exponential_decay(0.5, global_step, num_epoch * subset_size // batch_size, 0.96)
        optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)

In [27]:
with tf.Session(graph=graph, config=tf.ConfigProto(allow_soft_placement=True)) as session:
    
    # log the quality of the model
    train_accuracy_log = tf.scalar_summary("train accuracy", model_accuracy)
    valid_accuracy_log = tf.scalar_summary("validation accuracy", model_accuracy)
    test_accuracy_log = tf.scalar_summary("test accuracy", model_accuracy)
    loss_log = tf.scalar_summary("loss", loss)
    
    # Merge all the summaries and write them out to /tmp/mnist_logs
    writer = tf.train.SummaryWriter("/tmp/udacity_logs", session.graph_def)
        
    for i in range(2, train_dataset.shape[0] // batch_size, 2):
        
        # Reset all the variables
        tf.initialize_all_variables().run()
        session.run(global_step.assign(0))
        session.run(subset_size.assign(i * batch_size))
        print("Re-initialized \n")
        
        # Extract a subset from the training dataset
        subset_data, subset_labels = random_subset(train_dataset, train_labels, i * batch_size)

        # Train with a subset of n_i batch
        for step in range(num_epoch * i):
            # Generate a minibatch.
            offset = (step * batch_size) % (subset_data.shape[0] - batch_size)
            batch_data = subset_data[offset:(offset + batch_size), :]
            batch_labels = subset_labels[offset:(offset + batch_size), :]
            # Feed it to the optimizer
            feed_dict = {input_data : batch_data, output_data : batch_labels}
            session.run(optimizer, feed_dict=feed_dict)
            
        # Log the performance with a downsampled training set
        print('\n==> with a training subset of %d batches' % i)
        print('= %d samples' % subset_data.shape[0])
        print('= %d %% of the full train dataset size' % (100 * subset_data.shape[0] // train_dataset.shape[0]))
        print('= %d steps of training' % step)
        
        #evaluate on the train set
        feed_dict = {input_data : subset_data, output_data : subset_labels}
        acc, acc_str = session.run([model_accuracy, train_accuracy_log], feed_dict=feed_dict)
        writer.add_summary(acc_str, subset_data.shape[0])
        print("Train set accuracy: %f" % acc)

        #evaluate on the validation set
        feed_dict = {input_data : valid_dataset, output_data : valid_labels}
        acc, acc_str = session.run([model_accuracy, valid_accuracy_log], feed_dict=feed_dict)
        writer.add_summary(acc_str, subset_data.shape[0])
        print("Validation set accuracy: %f" % acc)

        #evaluate on the test set
        feed_dict = {input_data : test_dataset, output_data : test_labels}
        acc, acc_str = session.run([model_accuracy, test_accuracy_log], feed_dict=feed_dict)
        writer.add_summary(acc_str, subset_data.shape[0])
        print("Test set accuracy: %f" % acc)

Re-initialized 


==> with a training subset of 2 batches
= 256 samples
= 0 % of the full train dataset size
= 39 steps of training
Train set accuracy: 0.769531
Validation set accuracy: 0.603391
Test set accuracy: 0.603391
Re-initialized 


==> with a training subset of 4 batches
= 512 samples
= 0 % of the full train dataset size
= 79 steps of training
Train set accuracy: 0.902344
Validation set accuracy: 0.710244
Test set accuracy: 0.710244
Re-initialized 


==> with a training subset of 6 batches
= 768 samples
= 0 % of the full train dataset size
= 119 steps of training
Train set accuracy: 0.942708
Validation set accuracy: 0.733621
Test set accuracy: 0.733621
Re-initialized 


==> with a training subset of 8 batches
= 1024 samples
= 0 % of the full train dataset size
= 159 steps of training
Train set accuracy: 0.958008
Validation set accuracy: 0.775158
Test set accuracy: 0.775158
Re-initialized 


==> with a training subset of 10 batches
= 1280 samples
= 0 % of the full train dataset

KeyboardInterrupt: 

---
Problem 3
---------
Introduce Dropout on the hidden layer of the neural network. Remember: Dropout should only be introduced during training, not evaluation, otherwise your evaluation results would be stochastic as well. TensorFlow provides `nn.dropout()` for that, but you have to make sure it's only inserted during training.

What happens to our extreme overfitting case?

---

In [25]:
batch_size = 128

input_size = image_size * image_size
hidden_size = 90
output_size = num_labels

num_steps = 6000

graph = tf.Graph()
with graph.as_default():

    # Generic placeholders : it can either run with ttrain / test / valid data
    input_data = tf.placeholder(tf.float32, shape=(None, input_size), name='input')
    output_data = tf.placeholder(tf.float32, shape=(None, output_size), name='output')

    # First layer : dense
    with tf.variable_scope('first_layer') as scope:
        layer_1_weights = tf.Variable(tf.truncated_normal([input_size, hidden_size]), name='layer_1_weights')
        layer_1_biases = tf.Variable(tf.zeros([hidden_size]), name='layer_1_biases')
        layer_1_logits = tf.matmul(input_data, layer_1_weights) + layer_1_biases
        layer_1_relu = tf.nn.relu(layer_1_logits, name='layer_1_relu')
        
    # Second layer : dense
    with tf.variable_scope('second_layer') as scope:
        layer_2_weights = tf.Variable(tf.truncated_normal([hidden_size, output_size]), name='layer_2_weights')
        layer_2_biases = tf.Variable(tf.zeros([output_size]), name='layer_2_biases')
        layer_2_logits = tf.matmul(layer_1_relu, layer_2_weights) + layer_2_biases
    
    # Dropout
    with tf.variable_scope('dropout') as scope:
        keep_prob = tf.placeholder("float")
        layer_2_drop = tf.nn.dropout(layer_2_logits, keep_prob)
    
    # Output = prediction
    with tf.variable_scope('prediction') as scope:
        model_output = tf.nn.softmax(layer_2_drop, name='model_prediction')
        model_accuracy = accuracy(model_output, output_data, name='model_accuracy')
    
    # Loss
    with tf.variable_scope('loss') as scope:
        loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(layer_2_drop, output_data), name='loss')
    
    # Optimizer with dynamic learning rate
    with tf.variable_scope('optimizer') as scope:
        global_step = tf.Variable(0, trainable=False)
        learning_rate = tf.train.exponential_decay(0.5, global_step, num_steps, 0.96)
        optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)

In [26]:
with tf.Session(graph=graph, config=tf.ConfigProto(allow_soft_placement=True)) as session:
    
    # log the quality of the model
    train_accuracy_log = tf.scalar_summary("train accuracy", model_accuracy)
    valid_accuracy_log = tf.scalar_summary("validation accuracy", model_accuracy)
    test_accuracy_log = tf.scalar_summary("test accuracy", model_accuracy)
    loss_log = tf.scalar_summary("loss", loss)
    
    # Merge all the summaries and write them out to /tmp/mnist_logs
    writer = tf.train.SummaryWriter("/tmp/udacity_logs", session.graph_def)
    tf.initialize_all_variables().run()
    print("Initialized")
    
    for step in range(1,num_steps+1):
        if (step % 100 == 0):
            print("==> step %d" % step)
            
            #evaluate on the train set
            feed_dict = {input_data : train_dataset, output_data : train_labels, keep_prob: 1.0}
            acc, acc_str = session.run([model_accuracy, train_accuracy_log], feed_dict=feed_dict)
            writer.add_summary(acc_str, step)
            print("Train set accuracy: %f" % acc)
            
            #evaluate on the validation set
            feed_dict = {input_data : valid_dataset, output_data : valid_labels, keep_prob: 1.0}
            acc, acc_str = session.run([model_accuracy, valid_accuracy_log], feed_dict=feed_dict)
            writer.add_summary(acc_str, step)
            print("Validation set accuracy: %f" % acc)
            
            #evaluate on the test set
            feed_dict = {input_data : test_dataset, output_data : test_labels, keep_prob: 1.0}
            acc, acc_str = session.run([model_accuracy, test_accuracy_log], feed_dict=feed_dict)
            writer.add_summary(acc_str, step)
            print("Test set accuracy: %f" % acc)
        else:
            # Generate a minibatch.
            offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
            batch_data = train_dataset[offset:(offset + batch_size), :]
            batch_labels = train_labels[offset:(offset + batch_size), :]
            feed_dict = {input_data : batch_data, output_data : batch_labels, keep_prob: 0.5}
            session.run(optimizer, feed_dict=feed_dict)

Initialized
==> step 100
Train set accuracy: 0.671033
Validation set accuracy: 0.747266
Test set accuracy: 0.747266
==> step 200
Train set accuracy: 0.665614
Validation set accuracy: 0.729106
Test set accuracy: 0.729106
==> step 300
Train set accuracy: 0.686409
Validation set accuracy: 0.747366
Test set accuracy: 0.747366
==> step 400
Train set accuracy: 0.693755
Validation set accuracy: 0.752283
Test set accuracy: 0.752283
==> step 500
Train set accuracy: 0.706469
Validation set accuracy: 0.767633
Test set accuracy: 0.767633
==> step 600
Train set accuracy: 0.724352
Validation set accuracy: 0.783586
Test set accuracy: 0.783586
==> step 700
Train set accuracy: 0.725138
Validation set accuracy: 0.803853
Test set accuracy: 0.803853
==> step 800
Train set accuracy: 0.726638
Validation set accuracy: 0.806160
Test set accuracy: 0.806160
==> step 900
Train set accuracy: 0.734261
Validation set accuracy: 0.799940
Test set accuracy: 0.799940
==> step 1000
Train set accuracy: 0.738571
Validatio

---
Problem 4
---------

Try to get the best performance you can using a multi-layer model! The best reported test accuracy using a deep network is [97.1%](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html?showComment=1391023266211#c8758720086795711595).

One avenue you can explore is to add multiple layers.

Another one is to use learning rate decay:

    global_step = tf.Variable(0)  # count the number of steps taken.
    learning_rate = tf.train.exponential_decay(0.5, step, ...)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
 
 ---


In [37]:
batch_size = 128

input_size = image_size * image_size
hidden_1_size = 180
hidden_2_size = 40
output_size = num_labels

num_steps = 6000

graph = tf.Graph()
with graph.as_default():

    # Generic placeholders : it can either run with ttrain / test / valid data
    input_data = tf.placeholder(tf.float32, shape=(None, input_size), name='input')
    output_data = tf.placeholder(tf.float32, shape=(None, output_size), name='output')

    # First layer : dense
    with tf.variable_scope('first_layer') as scope:
        layer_1_weights = tf.Variable(tf.truncated_normal([input_size, hidden_1_size]), name='layer_1_weights')
        layer_1_biases = tf.Variable(tf.zeros([hidden_1_size]), name='layer_1_biases')
        layer_1_logits = tf.matmul(input_data, layer_1_weights) + layer_1_biases
        layer_1_relu = tf.nn.relu(layer_1_logits, name='layer_1_relu')
        
    # Second layer : dense
    with tf.variable_scope('second_layer') as scope:
        layer_2_weights = tf.Variable(tf.truncated_normal([hidden_1_size, hidden_2_size]), name='layer_2_weights')
        layer_2_biases = tf.Variable(tf.zeros([hidden_2_size]), name='layer_2_biases')
        layer_2_logits = tf.matmul(layer_1_relu, layer_2_weights) + layer_2_biases
        layer_2_relu = tf.nn.relu(layer_2_logits, name='layer_2_relu')
    
    # Dropout
    with tf.variable_scope('dropout') as scope:
        keep_prob = tf.placeholder("float")
        layer_2_drop = tf.nn.dropout(layer_2_relu, keep_prob)
        
    # Third layer : dense
    with tf.variable_scope('third_layer') as scope:
        layer_3_weights = tf.Variable(tf.truncated_normal([hidden_2_size, output_size]), name='layer_3_weights')
        layer_3_biases = tf.Variable(tf.zeros([output_size]), name='layer_3_biases')
        layer_3_logits = tf.matmul(layer_2_drop, layer_3_weights) + layer_3_biases
    
    # Output = prediction
    with tf.variable_scope('prediction') as scope:
        model_output = tf.nn.softmax(layer_3_logits, name='model_prediction')
        model_accuracy = accuracy(model_output, output_data, name='model_accuracy')
    
    # Loss
    with tf.variable_scope('loss') as scope:
        loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(layer_3_logits, output_data), name='loss')
    
    # Optimizer with dynamic learning rate
    with tf.variable_scope('optimizer') as scope:
        global_step = tf.Variable(0, trainable=False)
        learning_rate = tf.train.exponential_decay(0.5, global_step, num_steps, 0.96)
        optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)

In [38]:
with tf.Session(graph=graph, config=tf.ConfigProto(allow_soft_placement=True)) as session:
    
    # log the quality of the model
    train_accuracy_log = tf.scalar_summary("train accuracy", model_accuracy)
    valid_accuracy_log = tf.scalar_summary("validation accuracy", model_accuracy)
    test_accuracy_log = tf.scalar_summary("test accuracy", model_accuracy)
    loss_log = tf.scalar_summary("loss", loss)
    
    # Merge all the summaries and write them out to /tmp/mnist_logs
    writer = tf.train.SummaryWriter("/tmp/udacity_logs", session.graph_def)
    tf.initialize_all_variables().run()
    print("Initialized")
    
    for step in range(1,num_steps+1):
        if (step % 100 == 0):
            print("==> step %d" % step)
            
            #evaluate on the train set
            feed_dict = {input_data : train_dataset, output_data : train_labels, keep_prob: 1.0}
            acc, acc_str = session.run([model_accuracy, train_accuracy_log], feed_dict=feed_dict)
            writer.add_summary(acc_str, step)
            print("Train set accuracy: %f" % acc)
            
            #evaluate on the validation set
            feed_dict = {input_data : valid_dataset, output_data : valid_labels, keep_prob: 1.0}
            acc, acc_str = session.run([model_accuracy, valid_accuracy_log], feed_dict=feed_dict)
            writer.add_summary(acc_str, step)
            print("Validation set accuracy: %f" % acc)
            
            #evaluate on the test set
            feed_dict = {input_data : test_dataset, output_data : test_labels, keep_prob: 1.0}
            acc, acc_str = session.run([model_accuracy, test_accuracy_log], feed_dict=feed_dict)
            writer.add_summary(acc_str, step)
            print("Test set accuracy: %f" % acc)
        else:
            # Generate a minibatch.
            offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
            batch_data = train_dataset[offset:(offset + batch_size), :]
            batch_labels = train_labels[offset:(offset + batch_size), :]
            feed_dict = {input_data : batch_data, output_data : batch_labels, keep_prob: 0.5}
            session.run(optimizer, feed_dict=feed_dict)

Initialized
==> step 100
Train set accuracy: 0.101351
Validation set accuracy: 0.100231
Test set accuracy: 0.100231
==> step 200
Train set accuracy: 0.101351
Validation set accuracy: 0.100231
Test set accuracy: 0.100231
==> step 300
Train set accuracy: 0.101279
Validation set accuracy: 0.100130
Test set accuracy: 0.100130
==> step 400
Train set accuracy: 0.101037
Validation set accuracy: 0.099428
Test set accuracy: 0.099428
==> step 500
Train set accuracy: 0.101263
Validation set accuracy: 0.100130
Test set accuracy: 0.100130
==> step 600
Train set accuracy: 0.101351
Validation set accuracy: 0.100231
Test set accuracy: 0.100231
==> step 700
Train set accuracy: 0.101186
Validation set accuracy: 0.099930
Test set accuracy: 0.099930
==> step 800
Train set accuracy: 0.101279
Validation set accuracy: 0.100130
Test set accuracy: 0.100130
==> step 900
Train set accuracy: 0.101212
Validation set accuracy: 0.099829
Test set accuracy: 0.099829
==> step 1000


KeyboardInterrupt: 